In [1]:
from PIL import Image
import os
import numpy as np

%cd '/Users/juanmeriles/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/FogLater'

/Users/juanmeriles/Documents/GitHub/FogLater


/Users/juanmeriles/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
#Baseline Image
day = '20241018'
pathToImage = "../../data/day/"
files = os.listdir(pathToImage+day)
brightestImage = 0
brightestImageIndex = 0
brightest = 0
count = 0 

for file in files:
    with Image.open(pathToImage+day+'/'+file) as im:
        pixels = list(im.getdata())
        red, green, blue = im.split()
        brightness = np.average(red)
        if brightness > brightest:
            brightestImage = file
            brightestImageIndex = count
            brightest = brightness
    count += 1

print(f"The brightest image is {file} with an average pixel value of {brightness}")
with Image.open(pathToImage+day+'/'+files[brightestImageIndex]) as im:
    im.show()

The brightest image is 20241018.185117.png with an average pixel value of 60.36442565917969


In [11]:
pathToImage = "../../data/day/"
dateList = ['20241012','20241013','20241014','20241015','20241016','20241017','20241018']
#dateList = ['20241012']
print(os.getcwd())
allImages = []
cornerPixels = [0,511,512*511,512*512]

with Image.open(pathToImage+day+'/'+files[brightestImageIndex]) as controlIm:
    red, green, blue = im.split()
    controlIntensity = red.getdata()

for day in dateList:
    
    files = os.listdir(pathToImage+day)
    for file in files:
        brightDiff = 0
        with Image.open(pathToImage+day+'/'+file) as im:
            pixels = list(im.getdata())
            allImages.append(list(im.getdata()))
            red, green, blue = im.split()
            custom_img = Image.new(im.mode, im.size)
            custom_pixels = pixels
            imageIntensity = red.getdata()

        count = 1
        for i in range(len(imageIntensity)):
            if imageIntensity[i] < controlIntensity[i]:
                brightDiff += controlIntensity[i] - imageIntensity[i]
                count += 1

        brightDiff = int(brightDiff / count)
        #print(brightDiff)
        custom_pixels = [(r+brightDiff, g+brightDiff, b+brightDiff) for (r, g, b) in pixels]
        custom_img.putdata(custom_pixels)
        custom_img.save('../../BrightImages/' + file)
        #custom_img.show()
        


/Users/juanmeriles/Documents/GitHub/FogLater


Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt


KeyboardInterrupt: 

In [63]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

day = '20241017'
pathToImage = "../../data/day/"
files = os.listdir(pathToImage + day)
files.sort()
kernel = None

background_subtractor = cv2.createBackgroundSubtractorMOG2()

for file in files:
    with Image.open(pathToImage+day+'/'+file) as Im:
        cv2.startWindowThread()
        frame = np.array(Im)
        foreground_mask = background_subtractor.apply(frame);
        _, foreground_mask = cv2.threshold(foreground_mask, 250, 255, cv2.THRESH_BINARY)

        foreground_mask = cv2.erode(foreground_mask, kernel, iterations = 1)
        foreground_mask = cv2.dilate(foreground_mask, kernel, iterations = 2)

        contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        frameCopy = Im
         # We need to be sure about the area of the contours i.e. it should be higher than 400 to reduce the noise.
        for cnt in contours:
            foregroundPart = cv2.bitwise_and(frame, frame, mask=foreground_mask)

            stacked_frame = np.hstack((frame, foregroundPart, frameCopy))
            cv2.imwrite('../../data/maskedData/' + file,foregroundPart)

            cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked_frame, None, fx=0.5, fy=0.5))
            
            k = cv2.waitKey(1) & 0xff

            if k == ord('q'):
                break
    
cv2.destroyAllWindows()

